In [2]:
import tkinter as tk
from tkinter import Label, Entry, Button, StringVar, OptionMenu
import pandas as pd
import pickle
import PySimpleGUI as sg
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

df_we_meta = pd.read_csv(r'C:\Users\dsalvatorelli\Desktop\Group project\df_we_meta.csv')
filename = r'C:\Users\dsalvatorelli\Desktop\Group project\d2v.model'        
model = pickle.load(open(filename, 'rb'))

root = tk.Tk()
root.title('Wine Recommendation System')
root.geometry("1024x768")

metadata = df_we_meta['metadata']
metadata.to_list()  
tagged_documents = []
for i, doc in enumerate(metadata):
    tagged = TaggedDocument(doc, [i])
    tagged_documents.append(tagged)   
embedding = []
for i in range(0,len(tagged_documents)):
    tagged = model.docvecs[i]
    embedding.append(tagged)
    
cosine_sim_matrix_emb = cosine_similarity(embedding, embedding)

def recommend_meta_we(title, cosine_sim = cosine_sim_matrix_emb):
    mapping = pd.Series(df_we_meta["title"])
    recommended_wine = []
        
    idx = mapping[mapping == title].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:22].index)
        
    for i in top_10_indices:
        recommended_wine.append(df_we_meta['title'][i])

    pd.set_option('display.max_colwidth', None)
    score_meta = pd.DataFrame(list(zip(recommended_wine)),
                columns =[' '])
    return score_meta

options =  df_we_meta['title'].to_list()

clicked = StringVar()
clicked.set(options[0])


def show():
    global myLabel
    result = recommend_meta_we(clicked.get())
    myLabel = Label(root, text=result, font=('Helvetica', 12), justify="left", border=50, fg='black')
    myLabel.pack()

def mydelete():
    myLabel.destroy()

drop = OptionMenu(root, clicked, *options)
drop.pack()

myButton = Button(root, text="Recommend", font=('Helvetica', 12), command=show).pack()
deletebutton = Button(root, text="Delete", font=('Helvetica', 12), command=mydelete).pack()

root.mainloop()